**1. Using a .secrets Folder with a JSON File**

In [ ]:
import json

# Replace `path to the .json file` with the actual file path
json_file_path = r"path to the .json file"

with open(json_file_path, "r") as f:
    credentials = json.load(f)

# To access the username and password
username = credentials["username"]
password = credentials["password"]

In [7]:
credentials

{'username': 'your_username',
 'password': 'your_password',
 'email_receivers': ['xyz@gmail', 'abc@gmail']}

In [10]:
username = credentials["username"]
password = credentials["password"]

2. **Using Environment Variables**

Using Environment Variables

- Open the Windows search bar.
- Type in `control panel` and open it.
- Select S`ystem and Security`
- Choose `System`.
- Click on `Advanced system settings`
- Click on` Environment Variables`
- Under the "`User variables`" section, click `New...`
- Enter` Variable name` (e.g., username) and `Variable value` (e.g.,` your_username`).
- Repeat the process for the password by creating a new variable

In [ ]:
# To access Environment variables in Python
import os

# Username
username = os.getenv("username")

# Password
password = os.getenv("password")

# NB: If your vs code was open before creating the environment variables, you need to restart it for the changes to take effect

**3. .py file**

In [1]:
import securing_credentials

In [4]:
username = securing_credentials.username
password = securing_credentials.password

**Sending Emails**

In [11]:
import os
from email.message import EmailMessage
import ssl, smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [12]:
 html = '''
            <html>
                <head>
                    <style>
                        P{{font-family: Calibri; font-size: 17px;}}
                    </style>
                </head>
                <body>
                    <p>Hi, <br> This is a sample Email</p>                                 
                        <hr> 
                <P>Regards, <br> BCD</P>                        
                </body>
            </html>
            '''

In [15]:
# Stored as Environment Variables
email_from = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

# Email receivers
receivers = ','.join(credentials["email_receivers"])

email_message = MIMEMultipart()

email_message["From"] = email_from
email_message["To"] = receivers
email_message["Subject"] = f"This is a sample email" 
email_message.attach(MIMEText(html, "html"))

def attach_file_to_email(email, filename):
    """Attach a file identified by filename to email message"""
    with open(filename, 'rb') as file:
        file_data = MIMEApplication(file.read(), _subtype= filename.split(".")[1])
        file_data.add_header('Content-Disposition', 'attachment', filename=filename)
        email.attach(file_data)
    

file_names = ["Styled Excel.xlsx", "Styled Excel-2.xlsx"]

# Attach all the files
for file_name in file_names:
    attach_file_to_email(email_message, file_name)
    

# Email content
email_string = email_message.as_string()

# ssl context used to secure connection with the email server
context = ssl.create_default_context()

# Send email using Simple Mail Transfer Protocol (SMTP)
# SMTP server connection
with smtplib.SMTP("smtp.gmail.com", 587) as server:
    #(Extended Hello) Greet the server to initiate a conversation with the server
    server.ehlo()
    # Start a Transport Layer Security(TLS) connection with the server
    # SSL context created earlier is passed to ensure secure communication
    server.starttls(context=context)
    # After starting TLS, resend th ehlo command to inform the server that you now support encrypted communication
    server.ehlo()
    # Log into the gmail account using the credentials provided
    server.login(email_from, password)
    # Send the email
    server.sendmail(email_from, receivers, email_string)
    